In [ ]:
# Ejercicio I: Base de datos de una escuela en Colombia

import pandas as pd
import numpy as np

df_colegios = pd.read_csv('https://www.datos.gov.co/api/views/qijw-htwa/rows.csv?accessType=DOWNLOAD')

df_colegios

In [ ]:
# ¿Cuáles son las columnas? ¿Cuántas filas tiene?
df_colegios.columns

In [ ]:
# Seleccionen los colegios donde la propiedad de la planta física sea de una persona natural o comunidad religiosa (usar isin (investigar))
df_colegios[df_colegios["propiedad_Planta_Fisica"].isin(["PERSONA NATURAL",	"COMUNIDAD RELIGIOSA"])]

In [ ]:
# Contar los valores no nulos en la columna 'Discapacidades' imprime la info
df_colegios["discapacidades"].notnull().sum()

In [ ]:
# Definir los niveles que deseas verificar

# Filtrar los colegios que ofrecen los niveles deseados (usar isin)

# Imprimir la información de los colegios que ofrecen los niveles deseados
df_colegios[
    df_colegios["niveles"].str.contains("PREESCOLAR", na=False) &
    df_colegios["niveles"].str.contains("BÁSICA PRIMARIA", na=False) &
    df_colegios["niveles"].str.contains("BÁSICA SECUNDARIA", na=False)
]


Ejercicio II: PyEPH
PyEPH
La librería Pyeph tiene como objetivo facilitar el procesamiento en Python de las Encuesta Permanente de Hogares (eph) publicadas por INDEC de forma periódica. Está pensada como un espacio donde se nuclean y centralizan los cálculos vinculados a las mismas para posteriormente ser utilizadas en investigaciones, artículos, publicaciones, etc. Es una librería que hace principal hincapié en la transparencia metodológica utilizando licencias de código abierto y que promueve la colaboración de las comunidades de cientístas de datos, sociales, investigadores, desarrolladores, periodistas y demás curiosos.

# Instalamos la librería.

!pip install pyeph

In [ ]:
import pyeph

# Ejemplo: traemos el 2do trimestre del 2021
eph = pyeph.get(data="eph", year=2021, period=2, base_type='individual') # EPH individual
print(eph.columns.tolist())

Primer paso. Crear una variable llamada eph_2 que contenga las siguientes columnas para realizar el siguiente análisis:

Edad (CH_06), Sexo (CH_04), Ingreso de la ocupación principal (P21), Categoría ocupacional (CAT_OCUP), Condición de actividad (ESTADO), Ponderación (PONDERA)

Referencias:

Sexo:

01 = Varón
02 = Mujer
Categoría ocupacional (para ocupados y desocupados con ocupación anterior):

1 = Patrón
2 = Cuenta propia
3 = Obrero o empleado
4 = Trabajador familiar sin remuneración
9 = Ns/Nr
Estado - Condición de actividad:

0 = Entrevista individual no realizada
1 = Ocupado
2 = Descoupado
3 = Inactivo
4 = menor de 10 años

In [ ]:
#Filtramos por columnas
eph[['CH06', 'CH04', 'P21', 'CAT_OCUP', 'ESTADO', 'PONDERA']]

In [ ]:
#Renombramos columnas
eph = eph.rename(columns={
    'CH06': 'Edad',
    'CH04': 'Sexo',
    'P21': 'Ingreso de la ocupación principal',
    'CAT_OCUP': 'Categoría_ocupacional',
    'ESTADO': 'Condición_de_actividad',
    'PONDERA': 'Ponderación'
})
eph[["Edad","Sexo","Ingreso de la ocupación principal","Categoría_ocupacional","Condición_de_actividad","Ponderación"]]


La columna pondera representa un estimado poblacional de las personas que representa cada fila. Por lo tanto, para cada cálculo es necesario operar con esa columna. Por ejemplo, la fila 0 representa a 214 personas de 58 años de sexo masculino, que tienen $50.000 de ingreso de la ocupación principal, su categoría es cuentapropista y pertenecen al segmento de ocupados.

In [ ]:
#mirar las primeras filas
eph[["Edad","Sexo","Ingreso de la ocupación principal","Categoría_ocupacional","Condición_de_actividad","Ponderación"]].head(10)

In [ ]:
# Filtrar las filas donde la edad es mayor a 40 y cat_ocup está presente

eph = eph[["Edad","Sexo","Ingreso de la ocupación principal","Categoría_ocupacional","Condición_de_actividad","Ponderación"]]
eph = eph.query("Edad > 40 and Categoría_ocupacional != ''")
# Imprimir el resultado
eph


In [ ]:
# Contar la cantidad de hombres y mujeres
hombres = eph.query("Sexo == 1").shape[0]  #shape[0] cuenta cuántos hombres hay (cantidad de filas).
mujeres = eph.query("Sexo == 2").shape[0]
# Imprimir el resultado con etiquetas

print(hombres)
print(mujeres)


In [ ]:
# Filtrar por mujeres
mujeres = eph.query("Sexo == 2")

# Calcular la suma de ponderaciones para mujeres ocupadas y desocupadas
ocupadas = mujeres.query("Condición_de_actividad == 1")
desocupadas = mujeres.query("Condición_de_actividad == 2")

ocupadas = ocupadas["Ponderación"].sum()
desocupadas = desocupadas["Ponderación"].sum()

# Obtener la tasa de desocupación
tasa = desocupadas / (ocupadas + desocupadas)

# Imprimir resultados
print("Ocupadas:", ocupadas)
print("Desocupadas:", desocupadas)
print("Tasa de desocupación femenina:", round(tasa * 100, 2), "%")



In [ ]:
def calcular_tasa_desocupacion(dataframe, genero):
    """
    Calcula la tasa de desocupación para un género específico en el período seleccionado.

    Parameters:
    - dataframe: DataFrame que contiene los datos necesarios (eph_2 en este caso).
    - genero: Número que representa el género (1 para Varón, 2 para Mujer).

    Returns:
    - tasa_desocupacion: Tasa de desocupación para el género especificado.
    """
    # Filtrar por el género especificado
    genero_df = dataframe.query(f"Sexo == {genero}")

    # Calcular la suma de ponderaciones para el género ocupado y desocupado
    ocupados = genero_df.query("Condición_de_actividad == 1")
    desocupados = genero_df.query("Condición_de_actividad == 2")
    
    ocupados = ocupados["Ponderación"].sum()
    desocupados = desocupados["Ponderación"].sum()

    # Calcular la tasa de desocupación
    tasa_desocupacion = desocupados / (ocupados + desocupados)

    return tasa_desocupacion

# Calcular la tasa de desocupación para mujeres
tasa = calcular_tasa_desocupacion(eph, 2)

print(f"Tasa de desocupación para mujeres: {tasa}" )
